##### 필요 모듈 설치

In [3]:
!pip install pandas

     --------------------------------------- 11.0/11.0 MB 11.3 MB/s eta 0:00:00


In [16]:
!pip install requests

##### 라이브러리 불러오기

In [1]:
#오디오 전처리
import librosa #wav 파일을 불러와 파형을 가공하여 fft()나 mfcc등 다양항 형태로 변환가능
#소리는 기본적으로 특정 주파수를 가지는 sin함수들의 합입니다.
#특정 시간에 주파수 성분이 어떻게 구성되어 있는지 확인할 수 있는데요.
#음성 데이터 분석을 할 때 주파수 분석 기법을 많이 사용합니다. (파형 자체를 이용하기도 합니다!)
import librosa.display as dsp
from IPython.display import Audio

#데이터 전처리
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import os
from tqdm import tqdm 

#형태소 분석
from konlpy.tag import Kkma
from konlpy.tag import Hannanum
from konlpy.tag import Okt
import json #json파일에 있는 대사 txt불러오기

##### 음성데이터 dataset

In [2]:
#아날로그 데이터로 되어있는 음성 데이터를 디지털 신호로 변환
#librosa 라이브러리를 사용해 음성 데이터를 load

#예시로 음성데이터 load
elsound, sr= librosa.load('./voice/노인남여_노인대화43_F_happyjoy9923_76_전라_실내_48250.wav', sr=16000)
print('sample_rate:', sr, ',audio shape:', elsound.shape)
print('length:', elsound.shape[0]/float(sr),'secs')
#sampling rate의 의미는 초당 16000개(16000Hz 주파수)의 샘플을 가지고 있는 데이터라는 의미입니다.
#(1초에 음성 신호를 16000번 sampling)
#16000Hz으로 설정한 이유는 사람의 목소리는 대부분 16000Hz 안에 포함된다고 합니다.

sample_rate: 16000 ,audio shape: (152396,)
length: 9.52475 secs


In [3]:
#음성데이터로 데이터프레임 생성하기.
def train_dataset():
    folder= "./el_voice/"
    dataset= []
    for file in tqdm(os.listdir(folder),colour='green'): #tqdm은 진행progress보여주는 함수, os는 디렉토리불러오기
        if 'wav' in file:
            voice_file_path= os.path.join(folder,file) #os.path.join 폴더와 파일 경로 합치기
            data,sr= librosa.load(voice_file_path,sr=16000) #파일별로 하나씩 librosa로 load
            f_name= str(file) #지역명을 파일이름에서 뽑아내기위해 파일명 string으로 변환
            if "수도권" in str(file): #만약 파일이름에 수도권이 있다면
                 soodo_label= "수도권" #수도권 라벨이라 생성하고
                 dataset.append([data,soodo_label]) #데이터셋에 넣기
                 
            else:   #이외에
                gibang_label= f_name[str(file).find('_실')-2:str(file).find('_실')] #파일이름의 (_실) 앞 2글자부터 (_실) -1까지
                if "도권" in gibang_label: #하지만 (도권)이 들어가있다면
                    gibang_label.remove("도권") #(도권)은 빼라
                dataset.append([data,gibang_label]) #데이터셋에 넣기
            
            

    print(dataset)
    return pd.DataFrame(dataset,columns=['data','label'])
    #if region_label= '수도권':
    

In [38]:
voice_wav= train_dataset() #이 데이터셋을 voice_wav라 부르겠다

100%|██████████| 6038/6038 [00:06<00:00, 907.25it/s] 


[[array([-6.1035156e-05,  3.0517578e-05, -3.0517578e-05, ...,
       -2.4719238e-03, -2.3193359e-03, -2.1057129e-03], dtype=float32), '충청'], [array([-6.1035156e-05,  3.0517578e-05, -3.0517578e-05, ...,
        3.8757324e-03,  3.9672852e-03,  3.9062500e-03], dtype=float32), '충청'], [array([-6.1035156e-05,  3.0517578e-05, -3.0517578e-05, ...,
        3.4484863e-03,  3.3569336e-03,  3.4179688e-03], dtype=float32), '충청'], [array([-6.1035156e-05,  3.0517578e-05, -3.0517578e-05, ...,
       -3.6621094e-03, -3.7231445e-03, -4.1198730e-03], dtype=float32), '충청'], [array([-6.1035156e-05,  3.0517578e-05, -3.0517578e-05, ...,
        5.2185059e-03,  5.1574707e-03,  5.4016113e-03], dtype=float32), '충청'], [array([-6.1035156e-05,  3.0517578e-05, -3.0517578e-05, ...,
        1.0070801e-03,  1.0070801e-03,  1.6784668e-03], dtype=float32), '충청'], [array([-6.1035156e-05,  3.0517578e-05, -3.0517578e-05, ...,
       -6.1035156e-05,  0.0000000e+00,  0.0000000e+00], dtype=float32), '충청'], [array([-6.1035156e

In [39]:
voice_wav

,data,label
0,"[-6.1035156e-05, 3.0517578e-05, -3.0517578e-05...",충청
1,"[-6.1035156e-05, 3.0517578e-05, -3.0517578e-05...",충청
2,"[-6.1035156e-05, 3.0517578e-05, -3.0517578e-05...",충청
3,"[-6.1035156e-05, 3.0517578e-05, -3.0517578e-05...",충청
4,"[-6.1035156e-05, 3.0517578e-05, -3.0517578e-05...",충청
...,...,...
3014,"[6.1035156e-05, 3.0517578e-05, 3.0517578e-05, ...",전라
3015,"[6.1035156e-05, 3.0517578e-05, 3.0517578e-05, ...",전라
3016,"[6.1035156e-05, 3.0517578e-05, 3.0517578e-05, ...",전라
3017,"[6.1035156e-05, 3.0517578e-05, 3.0517578e-05, ...",전라


In [40]:
voice_wav['label'].unique() #라벨에 들어간 종류

array(['충청', '수도권', '경상', '전라'], dtype=object)

In [41]:
#지역마다 번호 나타내는 컬럼 추가


voice_wav['code']= voice_wav['label'].copy() 
#새로운칼럼 생성 라벨 컬럼복제


#실험
#if voice_wav['label'] = "수도권": 1, if voice_wav['label'] = "경상": 2, if voice_wav['label'] = "전라": 3, if voice_wav['label'] = "충청": 4
#voice_wav[(voice_wav['code'] == '수도권')] = '1'

In [42]:
#apply함수로 code내 데이터 변경
def loc_code(region):
    if region == '수도권':
        return 0
    elif region == '경상':
        return 1
    elif region == '전라':
        return 2
    elif region == '충청':
        return 3
    
voice_wav['code']= voice_wav['code'].apply(loc_code)
voice_wav
    

,data,label,code
0,"[-6.1035156e-05, 3.0517578e-05, -3.0517578e-05...",충청,3
1,"[-6.1035156e-05, 3.0517578e-05, -3.0517578e-05...",충청,3
2,"[-6.1035156e-05, 3.0517578e-05, -3.0517578e-05...",충청,3
3,"[-6.1035156e-05, 3.0517578e-05, -3.0517578e-05...",충청,3
4,"[-6.1035156e-05, 3.0517578e-05, -3.0517578e-05...",충청,3
...,...,...,...
3014,"[6.1035156e-05, 3.0517578e-05, 3.0517578e-05, ...",전라,2
3015,"[6.1035156e-05, 3.0517578e-05, 3.0517578e-05, ...",전라,2
3016,"[6.1035156e-05, 3.0517578e-05, 3.0517578e-05, ...",전라,2
3017,"[6.1035156e-05, 3.0517578e-05, 3.0517578e-05, ...",전라,2


In [43]:
voice_wav.head()

,data,label,code
0,"[-6.1035156e-05, 3.0517578e-05, -3.0517578e-05...",충청,3
1,"[-6.1035156e-05, 3.0517578e-05, -3.0517578e-05...",충청,3
2,"[-6.1035156e-05, 3.0517578e-05, -3.0517578e-05...",충청,3
3,"[-6.1035156e-05, 3.0517578e-05, -3.0517578e-05...",충청,3
4,"[-6.1035156e-05, 3.0517578e-05, -3.0517578e-05...",충청,3


In [33]:
voice_wav['code'].unique() #코드에 들어간 종류

array([3, 0, 1, 2], dtype=int64)

In [48]:
#code컬럼으로 오름차순으로 정렬
voice_wav.sort_values(by='code',axis=0,ascending= True,inplace=True)

In [49]:
voice_wav.head()

,data,label,code
1509,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",수도권,0
1342,"[0.00091552734, 0.00048828125, 0.0006713867, 0...",수도권,0
1341,"[0.00091552734, 0.00048828125, 0.0006713867, 0...",수도권,0
1340,"[0.00091552734, 0.00048828125, 0.0006713867, 0...",수도권,0
1339,"[0.00091552734, 0.00048828125, 0.0006713867, 0...",수도권,0


##### 말뭉치 설정

In [61]:
#대사수집 예시
import json
with open('./el_voice/노인남여_노인대화52_F_som020508_79_충청_실내_57254.json', encoding='utf-8') as t:
    script= json.load(t)

print(script['발화정보']['stt'])

작년에 좀 이상한 게 와서 그런데 지금은 많이 완쾌된 상태예요



In [97]:
import re

scpt_folder= ('./el_voice/')
scpt_lst= []
for file in tqdm(os.listdir(scpt_folder),colour='green'): #tqdm은 진행progress보여주는 함수, os는 디렉토리불러오기
        if 'json' in file:
            scrpt_file_path= os.path.join(scpt_folder,file) #os.path.join 폴더와 파일 경로 합치기
            
            with open(scrpt_file_path, encoding='utf-8') as t: #json파일 불러오기
                script= json.load(t) #변수에 저장
                txt=(script['발화정보']['stt'].replace('\r\n','')) #대사만 추출하고 끝에 문자 삭제
                rmve_bracket = "\(.*\)|\s-\s.*"  #괄호와 괄호안의 정보
                txt_1=re.sub(rmve_bracket,'',txt) #(변경 전/ 변경 후/ 적용할 텍스트)
            scpt_lst.append(txt_1) #변형된 대사들 리스트 저장

len(scpt_lst)


100%|██████████| 6038/6038 [00:02<00:00, 2794.25it/s]


3019

In [98]:
print(scpt_lst[405:467])

['나무들 키가 엄청 커서 그늘이 잘 지기 때문에 돗자리 펴기 좋아요', '거기에 누워서 쉬기도 하고 힐링도 하고 그러는 게 너무 좋더라고요', '그리고 거기는 옆으로 가면  곤충식물원 같은 게 연결되어있어요', '옆으로 이동하면 애기들이 되게 좋아하는 구경거리들이 많이 있어요', '체험공간이 잘 되어있다 보니까 학교에서 단체로도 많이 오더라고요', '현장체험학습으로 오기 정말 좋은 공간이라고 생각해요 정말 잘해놨어', '너무 잘해놔서 한번 보면 다음에 또 가고  만들어 둔 것 같아요', '꽃도 우리가 평소에 보지 못했던 신비롭고 예쁜 꽃들이 굉장히 많아요', '그래서 보면 정말 예쁜데 또 그 밑으로 물고기들이  다니고 있어요', '물고기들이 이렇게 막 다니는데  모양이  재미있기도 해요', '그리고 꽃 중에서  양귀비 꽃이 예쁘잖아요 그것도 굉장히  장관이더라고요', '다음에 물향기  가게 되면 지금보다 더 잘되어 있을 것 같아요', '그래서 다음에 또 한번 와서 꽃도 구경하고 그러면 참 좋겠다고 생각합니다', '외도는 휴양지보다는 관광지라고 생각했는데 휴양지로도 괜찮은 것 같아', '회사에서 단체로 갔었는데 진짜 거기는 너무 아름답고 특별한 곳이야', '특별한 게 뭐나면 이 좁은 바위틈 사이로 배가 지나가게 되어있어', '그래서 안개가 끼는 날씨가 안 좋은 날에는 거기를 들어갈 수가 없어', '우리가 갔을 때 거기를 통과해서 지나갔는데 어떤 부부가 개발한 거래', '산 전체를 나무로 가지고 모형을 만들어 놓은  동화 속 같아요', '산도 꽤 큰데 산 전체를 어떻게 이렇게 만들었을까 궁금했어요', '우리가 간 날에는 비가 좀 오긴 했지만 많이 안 와서 다행이에요', '구경할 것은 다 봤는데도 여유롭게 둘러보지 못한 것 같아요', '비가 후두둑 떨어지는데 좋더라고요 예쁘게 사진도 찍고 같이 놀았어요', '나무를 깎아서 그네도 만들어 놓고  길길마다 그렇게 만들어 놨더라고요', '산을 한바퀴 돌면서 사진도 찍고 그러니까 시간이 꽤 걸리더라고요', '꼭 단체가 아